# Confounding Example: Finding causal effects from observed data

Suppose you are given some data with treatment and outcome. Can you determine whether the treatment causes the outcome, or the correlation is purely due to another common cause?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import dowhy
from dowhy import CausalModel
import dowhy.datasets, dowhy.plotter

# Config dict to set the logging level
import logging.config
DEFAULT_LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'loggers': {
        '': {
            'level': 'INFO',
        },
    }
}

logging.config.dictConfig(DEFAULT_LOGGING)

## Let's create a mystery dataset for which we need to determine whether there is a causal effect.

Creating the dataset. It is generated from either one of two models:
* **Model 1**: Treatment does cause outcome. 
* **Model 2**: Treatment does not cause outcome. All observed correlation is due to a common cause.

In [ ]:
rvar = 1 if np.random.uniform() >0.5 else 0
data_dict = dowhy.datasets.xy_dataset(10000, effect=rvar, 
                                      num_common_causes=1, 
                                      sd_error=0.2)                                               
df = data_dict['df']                                                                                 
print(df[["Treatment", "Outcome", "w0"]].head())
                                                                                                                                                                                                         

In [ ]:
dowhy.plotter.plot_treatment_outcome(df[data_dict["treatment_name"]], df[data_dict["outcome_name"]],
                             df[data_dict["time_val"]])                                                       

## Using DoWhy to resolve the mystery: *Does Treatment cause Outcome?*
### STEP 1: Model the problem as a causal graph
Initializing the causal model.

In [ ]:
model= CausalModel(                                                                                                                      
        data=df,                                                                                                                         
        treatment=data_dict["treatment_name"],                                                                                           
        outcome=data_dict["outcome_name"],                                                                                               
        common_causes=data_dict["common_causes_names"],                                                                                  
        instruments=data_dict["instrument_names"])                                                                                       
model.view_model(layout="dot")                                                                                                                                                                                                                                             

Showing the causal model stored in the local file "causal_model.png"

In [ ]:
from IPython.display import Image, display
display(Image(filename="causal_model.png"))

### STEP 2: Identify causal effect using properties of the formal causal graph
Identify the causal effect using properties of the causal graph.

In [ ]:
df

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)
                                                                                                                                                                                                                                     

from dowhy.causal_graph import CausalGraph

graph = CausalGraph(
            data_dict["treatment_name"],
            data_dict["outcome_name"],
            common_cause_names=data_dict["common_causes_names"],
            observed_node_names=df.columns.tolist(),
)

nx_graph = graph._graph

In [ ]:
from dowhy.causal_identifier import identify_effect

identified_estimand = identify_effect(nx_graph, action_nodes=data_dict["treatment_name"], outcome_nodes=data_dict["outcome_name"], observed_nodes=list(graph.get_all_nodes(include_unobserved=False)))

### STEP 3: Estimate the causal effect

Once we have identified the estimand, we can use any statistical method to estimate the causal effect. 

Let's use Linear Regression for simplicity.

In [ ]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print("Causal Estimate is " + str(estimate.value))

# Plot Slope of line between treamtent and outcome =causal effect                                                                                                 
dowhy.plotter.plot_causal_effect(estimate, df[data_dict["treatment_name"]], df[data_dict["outcome_name"]])

### Checking if the estimate is correct

In [ ]:
print("DoWhy estimate is " + str(estimate.value)) 
print ("Actual true causal effect was {0}".format(rvar))

### Step 4: Refuting the estimate

We can also refute the estimate to check its robustness to assumptions (*aka* sensitivity analysis, but on steroids). 

### Adding a random common cause variable

In [ ]:
from dowhy.causal_refuters import refute_estimate

refuter_class = dowhy.causal_refuters.get_class_object("random_common_cause")

refuters = list()
refuters.append(refuter_class)

res_random = refute_estimate(data=df, 
                             treatment_name=data_dict["treatment_name"], 
                             target_estimand=identified_estimand,
                             outcome_name=data_dict["outcome_name"], 
                             identified_estimand=identified_estimand, 
                             estimate=estimate,
                             refuters=refuters)
print(res_random)

### Replacing treatment with a random (placebo) variable

In [ ]:
refuter_class = dowhy.causal_refuters.get_class_object("placebo_treatment_refuter")

refuters = list()
refuters.append(refuter_class)

res_placebo = refute_estimate(data=df, 
                             treatment_name=data_dict["treatment_name"], 
                             target_estimand=identified_estimand,
                             outcome_name=data_dict["outcome_name"], 
                             identified_estimand=identified_estimand, 
                             estimate=estimate,
                             refuters=refuters,
                             placebo_type="permute")
print(res_placebo)


### Removing a random subset of the data

In [ ]:
refuter_class = dowhy.causal_refuters.get_class_object("data_subset_refuter")

refuters = list()
refuters.append(refuter_class)

res_subset = refute_estimate(data=df, 
                             treatment_name=data_dict["treatment_name"], 
                             target_estimand=identified_estimand,
                             outcome_name=data_dict["outcome_name"], 
                             identified_estimand=identified_estimand, 
                             estimate=estimate,
                             refuters=refuters,
                             subset_fraction=0.9)
print(res_subset)

As you can see, our causal estimator is robust to simple refutations.